In [101]:
import warnings
warnings.filterwarnings("ignore")

import sys
import glob
import time
import gc
import copy
import argparse
import pickle
from threading import Thread
from os.path import splitext, basename, join, isfile, dirname, realpath, exists
from datetime import datetime

import pandas as pd
import numpy as np

sys.path.append('/home/alexandrearaujo/libraries/faiss/')
import pandas as pd
import numpy as np
import faiss

%matplotlib inline
import matplotlib.pyplot as plt

In [5]:
def pickle_load(path):
  """function to load pickle object"""
  with open(path, 'rb') as f:
    return pickle.load(f, encoding='latin1')

In [102]:
submit = pd.read_csv('/home/alexandrearaujo/kaggle/landmark-retrieval-challenge/submit/18_dir_dbe/submit_2018-04-25_15.45.35_dir_full_index_qe_2.csv.gz')

In [31]:
query = submit['id'].values
query_preds = submit['images'].values

In [12]:
index_query = pickle_load('/home/alexandrearaujo/kaggle/landmark-retrieval-challenge/submit/18_dir_dbe/idx.pkl')
train_filenames = pickle_load('/home/alexandrearaujo/kaggle/landmark-retrieval-challenge/submit/18_dir_dbe/filenames_index.dump')

In [49]:
inverse_train_filenames = dict(list(zip(train_filenames, list(range(len(train_filenames))))))

In [77]:
def print_query(name):
    print(name)
    path = '/media/hdd1/kaggle/landmark-retrieval-challenge/test_rescale/{}.jpg'.format(name)
    plt.imshow(cv2.imread(path))
    plt.show()
    
def print_index(names):
    names = names.split(' ')
    for name in names[:5]:
        path = '/media/hdd1/kaggle/landmark-retrieval-challenge/index_rescale/{}.jpg'.format(name)
        plt.imshow(cv2.imread(path))
        plt.show()

In [98]:
rerank_res = {}
start = datetime.now()
for i, x in enumerate(query):
    if not isinstance(query_preds[i], str):
        continue
    
    if i % 10000 == 0 and i != 0:
        elapsed = (datetime.now() - start).total_seconds()
        print('image {}/{} in {} secs'.format(i, len(query), elapsed))
        start = datetime.now()
        
    array_query_preds = set(query_preds[i].split(' '))
    new_list = []
    for rank, preds in enumerate(array_query_preds):
        ix = inverse_train_filenames[preds]
        prediction_from_preds = set(map(lambda x: str(train_filenames[x]), index_query[ix]))
        new_list.append((preds, len(array_query_preds.intersection(prediction_from_preds)), rank))
    new_prop = ' '.join([x[0] for x in sorted(new_list, key=lambda x: x[1], reverse=True)])
    rerank_res[x] = new_prop

image 10000/117703 in 27.977095 secs
image 20000/117703 in 26.894986 secs
image 30000/117703 in 26.656892 secs
image 40000/117703 in 26.898747 secs
image 50000/117703 in 27.788465 secs
image 60000/117703 in 27.80053 secs
image 70000/117703 in 27.723289 secs
image 80000/117703 in 27.781741 secs
image 90000/117703 in 27.99118 secs
image 100000/117703 in 28.468166 secs
image 110000/117703 in 29.301114 secs


In [100]:
submit = pd.read_csv('../data/sample_submission.csv')
submit['images'] = submit['id'].apply(lambda ix: rerank_res.get(ix, ''))
submit_filename = '../submit/18_dir_dbe/submit_{}_dir_full_index_qe_{}_rerank.csv.gz'.format(
            datetime.now().strftime('%Y-%m-%d_%H.%M.%S'), 2)
submit.to_csv(submit_filename, index=False, compression='gzip')

In [113]:
def sanitize(data):
  """ convert array to a c-contiguous float array """
  if data.dtype != 'float32':
    data = data.astype('float32')
  if not data.flags['C_CONTIGUOUS']:
    data = np.ascontiguousarray(data)
  return data

In [147]:
submit.iloc[1]

id                                         0001623c6d808702
images    1313121533ed3e26 5c4759d1c8749384 9faf835bdfe4...
Name: 1, dtype: object

In [148]:
path = '/media/hdd1/kaggle/landmark-retrieval-challenge/feature_test_rescale_delf_pkl/{}.pkl'.format('0001623c6d808702')
loc1, desc1 = pickle_load(path)

In [149]:
path = '/media/hdd1/kaggle/landmark-retrieval-challenge/feature_index_rescale_delf_pkl/{}.pkl'.format('1313121533ed3e26')
loc2, desc2 = pickle_load(path)

In [174]:
index = faiss.IndexFlatL2(40)
index.add(sanitize(desc2))
D, I = index.search(sanitize(desc1), 10)

In [175]:
ix = np.arange(0, 578).repeat(10).reshape(578, 10)

In [178]:
loc2[Ik[Dk <= 0.8]]
loc1[ix[Dk <= 0.8]]

array([[160.        , 320.        ],
       [  0.        ,  90.51054382],
       [160.        , 240.        ],
       [ 22.62763214, 158.39344788],
       [ 16.        , 144.        ],
       [192.        ,  96.        ],
       [160.        , 256.        ],
       [  0.        ,  96.        ],
       [  0.        ,  90.51053619],
       [112.        , 112.        ],
       [ 48.        , 224.        ],
       [452.55270386, 746.71191406],
       [448.        , 784.        ],
       [480.        , 784.        ],
       [464.        , 704.        ],
       [475.18032837, 769.33959961],
       [452.4887085 , 723.98193359],
       [160.        , 416.        ],
       [158.39344788, 429.92504883],
       [160.        , 320.        ],
       [160.        , 288.        ],
       [416.        , 352.        ],
       [384.        , 352.        ],
       [368.        , 448.        ],
       [362.04217529, 429.92504883],
       [407.29742432, 339.41452026],
       [158.39344788, 429.92504883],
 

limit_output extension: Maximum message size of 10000 exceeded with 11581 characters

In [185]:
df = pd.read_csv('../submit/18_dir_dbe/submit_2018-05-03_15.14.24_rerank.csv.gz')

In [184]:
df[~df['images'].isnull()]

,id,images


In [186]:
df

,id,images
0,000088da12d664db,ef2f7cd74e6be523 d67021ad4bd41e2d ae44ad2d954a...
1,0001623c6d808702,NaN
2,0001bbb682d45002,NaN
3,0002362830cfe3a3,NaN
4,000270c9100de789,NaN
5,0002b0fab5d3ccc4,NaN
6,000396be3c24830a,NaN
7,000506dc6ab3a40e,NaN
8,0005292fc4b005a3,NaN
9,0005456a82264bc8,NaN


In [204]:
src = np.random.random(size=(10, 2))
dst = np.random.random(size=(10, 2))

In [205]:
from skimage.measure import ransac
from skimage.transform import AffineTransform

In [206]:
# # estimate affine transform model using all coordinates
# model = AffineTransform()
# model.estimate(src, dst)

# robustly estimate affine transform model with RANSAC
model_robust, inliers = ransac((src, dst), AffineTransform, min_samples=3,
                               residual_threshold=2, max_trials=100)

In [207]:
inliers

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])